Dans ce notebook, on ne va chercher à faire des prédictions mais plutot à faire des régressions entre différentes variables afin de voir s'il est possible d'en expliquer certaines à partir d'autres. Concrètement, il s'agit de voir si le taux de souscription à un dépôt de long terme peut être expliquée par d'autres variables.

## 1. Import des librairies et chargement des données 

In [2]:
import pandas as pd
import numpy as np
from Preprocessing import preprocessing
import statsmodels.api as sm
from sklearn import linear_model

pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

In [3]:
df = pd.read_csv("bank-full.csv",  sep = ";")
df.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,no
1,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown,no
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown,no
3,47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,-1,0,unknown,no
4,33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1,-1,0,unknown,no


In [4]:
np.shape(df)

(45211, 17)

On a 45211 observations pour 17 variables
- La variable d'intérêt (cible) est "y", représentant le taux de souscription à un dépôt de long terme
- On observe 7 variables numériques: "age", "balance", "day", "duration", "campaign", "pdays" et "previous" (même si "day" pourrait être considéré comme une variable catégorielle)
- On observe 10 variables catégorielles:
    * 4 variables binaires: "default", "housing", "loan" et "y"
    * 6 variables avec strictement plus de 2 modalités: "job", "marital", "education", contact", "month" et "poutcome"

In [6]:
var_bin = ["default", "housing", "loan", "y"]
var_cat = ["job","marital", "education", "contact", "month", "poutcome"]

## 2. Regression linéaire sur le taux de souscription

On va vite se rendre compte qu'une régression linéaire n'est pas pertinente pour expliquer le comportement d'une variable binaire à partir d'une majorité de variables catégorielles mais on le fait quand même pour l'exercice.

Pour une regression linéaire, on utilise uniquement les variables numériques (ou les variables catégorielles ordinales).

In [5]:
#Encoding binaire grâce à la fonction factorize de pandas 
df["y_bin"] = pd.factorize(df["y"])[0]

X = df.drop(columns=["y_bin"]+var_bin+var_cat, axis = 1)
y = df.y_bin

### 2.1 Modèle scikit

In [13]:
model = linear_model.LinearRegression()
model.fit(X, y)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [14]:
print(model.intercept_, model.coef_)

-0.05036613824138704 [ 7.53699141e-04  4.25692230e-06 -5.72659963e-05  4.88198516e-04
 -3.19288543e-03  2.44627116e-04  7.85726699e-03]


On peut observer deux types de paramètres: 
- L'intercept correspond à la constante dans l'équation d'une régression linéaire.
- Les autres paramètres correspondent aux différents coefficients devant les variables prises en compte pour faire la régression.

On observe que les coefficients de la regression sont particulièrement faibles et donc que l'influence de chacune des variables sur la souscription est très réduite.

In [10]:
model.score(X, y)

0.1725236594584415

On observe également un faible R2 qui montre bien que la régressio linéaire n'est pas une bonne approche pour modéliser le comportement de la variable cible.

### 2.2 Modèle statmodels

In [21]:
model = sm.OLS(y, X)
results = model.fit()
# Avec  statsmodel, on a une sortie qui ressemble beaucoup à celle de R
print(results.summary())

                                 OLS Regression Results                                
Dep. Variable:                  y_bin   R-squared (uncentered):                   0.268
Model:                            OLS   Adj. R-squared (uncentered):              0.268
Method:                 Least Squares   F-statistic:                              2368.
Date:                Mon, 17 Feb 2020   Prob (F-statistic):                        0.00
Time:                        16:52:38   Log-Likelihood:                         -8584.3
No. Observations:               45211   AIC:                                  1.718e+04
Df Residuals:                   45204   BIC:                                  1.724e+04
Df Model:                           7                                                  
Covariance Type:            nonrobust                                                  
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------

On remarque que le R2 est légèrement meilleur avec statmodels même s'il reste assez faible. 

La colonne intéressante à regarder est "P>|t|" (cela s'appelle la p-value pour ceux qui veulent creuser) qui indique si les coefficients associés à chaque variable sont statistiquement significatifs. Si le nombre indiqué est supérieur à 0.05 (comme c'est le cas pour l'âge), cela signifie que le coefficient n'est pas statistiquement significatif. 

Même si les autres coefficients sont significatifs, on voit qu'ils sont très faibles. Ils influent donc peu sur le taux de souscription. De plus, l'interprétation de ces coefficients est assez difficile puisque la variable cible est binaire. Cependant, on pourrait par exemple dire qu'augmenter le nombre de contact avant campagne (previous) de 100 permet semble ajouter 78% de chance de souscription (ce qui semble malgré tout assez peu vraisemblable). De même, augmenter la durée des appels de 1000 secondes semblerait ajouter 50% de chance de souscription (ce qui va à l'encontre des analyses descriptives faite précédemment). D'où l'interêt de faire des analyses descriptives poussées ! 

## 3. Regression logistique sur le taux de souscription

On a créé un script "Preprocessing.py" qui permet de faire des transformations du dataset et notamment un one-hot encoding des variables catégorielles

In [25]:
from Preprocessing import preprocessing

df_prep = preprocessing(df, var_bin, var_cat)

In [26]:
df_prep.head()

,age,balance,day,duration,campaign,pdays,previous,y_bin,default_bin,housing_bin,loan_bin,job_blue-collar,job_entrepreneur,job_housemaid,job_management,job_retired,job_self-employed,job_services,job_student,job_technician,job_unemployed,job_unknown,marital_married,marital_single,education_secondary,education_tertiary,education_unknown,contact_telephone,contact_unknown,month_aug,month_dec,month_feb,month_jan,month_jul,month_jun,month_mar,month_may,month_nov,month_oct,month_sep,poutcome_other,poutcome_success,poutcome_unknown
0,58,2143,5,261,1,-1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1
1,44,29,5,151,1,-1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1
2,33,2,5,76,1,-1,0,0,0,1,1,0,1,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1
3,47,1506,5,92,1,-1,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1
4,33,1,5,198,1,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1


In [27]:
X = df_prep.drop(columns=["y_bin"], axis = 1)
y = df_prep.y_bin

### 3.1 Modèle scikit

In [64]:
model = linear_model.LogisticRegression(solver = "liblinear")
model.fit(X, y)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='liblinear', tol=0.0001, verbose=0,
                   warm_start=False)

Une fois le modèle entrainé, on peut récupérer les coefficients de la regression logistique grâce aux commandes suivantes. 

In [65]:
print(model.intercept_, model.coef_)

[-1.5192664] [[-6.99625226e-03  1.29742046e-05  6.46730418e-03  4.14691801e-03
  -9.35766160e-02 -6.58804239e-04  1.30657531e-02 -4.91783299e-02
  -7.15991280e-01 -4.02549136e-01 -3.84912936e-01 -3.43140758e-01
  -5.07890184e-01 -1.55123324e-01  3.17380084e-01 -2.93804475e-01
  -2.20018144e-01  3.03494646e-01 -1.60796272e-01 -1.89948114e-01
  -1.29123929e-01 -3.32330667e-01 -1.01011140e-01  5.64433834e-02
   2.13089524e-01  1.24442768e-01 -1.66701904e-01 -1.61163163e+00
  -8.22906872e-01  4.15657220e-01 -3.57545008e-01 -1.41075646e+00
  -9.90168248e-01  2.73716757e-01  1.50992561e+00 -5.46017058e-01
  -1.00696177e+00  7.49096857e-01  7.22775854e-01  9.10063932e-02
   2.04349706e+00 -3.03764694e-01]]


In [66]:
for i in range(len(X.columns)):
    print(X.columns[i], np.exp(model.coef_[0][i]))

age 0.9930281645417701
balance 1.0000129742887303
day 1.006488262349426
duration 1.0041555283711743
campaign 0.9106682430042564
pdays 0.9993414127252553
previous 1.0131514830619113
default_bin 0.9520113425040673
housing_bin 0.4887074259269507
loan_bin 0.6686134854216622
job_blue-collar 0.6805098814169571
job_entrepreneur 0.709538331462123
job_housemaid 0.6017638512692465
job_management 0.8563095674245043
job_retired 1.3735245275491184
job_self-employed 0.7454222259747062
job_services 0.8025042369736011
job_student 1.354584338970554
job_technician 0.8514655204820484
job_unemployed 0.8270020430472041
job_unknown 0.8788650418447425
marital_married 0.7172501124056774
marital_single 0.9039229635226524
education_secondary 1.0580667089955422
education_tertiary 1.2374954314179658
education_unknown 1.1325172029468622
contact_telephone 0.8464518978582137
contact_unknown 0.199561736606867
month_aug 0.4391532348588855
month_dec 1.5153663426140322
month_feb 0.6993912203678483
month_jan 0.2439586674

Ces coefficients peuvent être interprés de manière assez simple. Par exemple, une personne qui a déjà souscrit à un produit lors d'une campagne précédente (poutcome_success) a presque 8 fois plus de chances de souscrire, toute chose égale par ailleurs. De même, une personne contactée en Mars à 4 fois plus de chances de souscrire. 

Cependant, ces coefficients ne sont pas forcément significatifs, ce en quoi le modèle de statmodel nous donne plus d'informations 

### 3.2 Modèle statmodels

In [69]:
model = sm.Logit(y, X)
results = model.fit()
# Avec  statsmodel, on a une sortie qui ressemble beaucoup à celle de R
print(results.summary())

Optimization terminated successfully.
         Current function value: 0.240627
         Iterations 8
                           Logit Regression Results                           
Dep. Variable:                  y_bin   No. Observations:                45211
Model:                          Logit   Df Residuals:                    45169
Method:                           MLE   Df Model:                           41
Date:                Mon, 17 Feb 2020   Pseudo R-squ.:                  0.3332
Time:                        17:30:46   Log-Likelihood:                -10879.
converged:                       True   LL-Null:                       -16315.
Covariance Type:            nonrobust   LLR p-value:                     0.000
                          coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------
age                    -0.0179      0.002     -9.901      0.000      -0.021      -0.014
ba

On voit que le R2 pour la regression logistique est de 0.33 contre 0.27 pour la régression linéaire, ce qui est un peu mieux mais qui ne permet pas non plus de décrire entièrement la variable cible. 

De plus, on se rend compte que certains coefficients ne sont pas du tout significatifs tels que "day", "previous", "default_bin", "housing-bin", "job_student" ou encore "education_tertiary". En effet, leur p-value ( P>|z| ) est supérier à 0.05